In [44]:
import codecs
import os
import pickle

from tqdm.auto import tqdm
from utils import g_path, rawcount

In [254]:
# Is this file clean? Yes. What is missing from the clean file then?

texts_in_plain = {}
texts_in_clean = {}

for l in open("../data/clicked_docs.tsv", encoding="utf-8"):
    url = (l.strip().split("\t", maxsplit=1))[0].strip()
    try:
        text = (l.strip().split("\t", maxsplit=1))[1].strip()
    except:
        continue
    texts_in_plain[url] = text

for l in open("../data/clicked_docs_clean.tsv", encoding="utf-8"):
    url = (l.strip().split("\t", maxsplit=1))[0].strip()
    try:
        text = (l.strip().split("\t", maxsplit=1))[1].strip()
    except:
        continue
    texts_in_clean[url] = text

docs_in_plain = set(texts_in_plain)
docs_in_clean = set(texts_in_clean)

missing_docs = docs_in_plain.difference(docs_in_clean)
print(missing_docs)

# opposite? YUP.
missing_docs = docs_in_clean.difference(docs_in_plain)
print(missing_docs)

all_doc_ids = docs_in_clean.union(docs_in_plain)
both = docs_in_clean.intersection(docs_in_plain)

set()
{'https://www.telegraph.co.uk/finance/2815755/Timeline-The-sub-prime-mortgage-crisis.html', 'https://www.businesscycle.com/', 'https://www.ethics.net/', 'https://shop.grossdomesticproduct.com/'}


In [255]:
prefer_clean = {
    "https://www.amboss.com/us/knowledge/Hearing_loss",
    "https://www.medicaleconomics.com/medical",
    "https://ethics.sc.gov/",
    "https://worldview.stratfor.com/article/global-market-brief-subprime-crisis-goes-europe",
    "https://commons.wikimedia.org/wiki/Category:Business_cycle"
}
prefer_plain = {
    "https://www.webmd.com/ibs/guide/irritable-bowel-syndrome-treatment-care",
    "https://www.dictionary.com/browse/biogenetic",
    "https://www.starkey.com/blog/articles/2016/12/tinnitus-tied-to-hearing-loss",
    "https://www.businesscycle.com/ecri-news-events/news",
    "https://www.dictionary.com/browse/ethic",
    "https://www.health.harvard.edu/newsletter_article/Noise-induced_hearing_loss",
    "https://www.nhs.uk/conditions/irritable-bowel-syndrome-ibs/symptoms/",
    "https://www.mdpi.com/1660-4601/17/4/1201/htm",
    "http://www.moentcenter.com/library/4036/Noise-InducedHearingLossInChildren.html",
    "https://ethicsunwrapped.utexas.edu/glossary",
    "https://www.webmd.com/ibs/guide/irritable-bowel-syndrome-symptoms-types",
    "https://www.khanacademy.org/economics-finance-domain/macroeconomics",
    "https://www.dictionary.com/browse/gastroenterology",
    "https://www.thoughtco.com/phases-of-the-business-cycle-1146345",
    "https://www.learnreligions.com/virtue-ethics-morality-and-character-249866",
    "https://www.webmd.com/ibs/qa/what-triggers-irritable-bowel-syndrome",
    "https://www.economywatch.com/us-subprime/impact-europe.html",
    "https://study.com/academy/practice/quiz-worksheet-the-business-cycle-in-economics.html",
    "https://en.wiktionary.org/wiki/gastroenterology",
    "http://www.nationalacademies.org/hmd/Activities/Veterans/MilitaryHearingLoss.aspx",
    "https://www.wikem.org/wiki/Irritable_bowel_syndrome",
    "https://megsmenopause.com/2020/01/29/ibs/",
    "https://www.amazon.com/Austrian-School-Business-Cycle-Theory/dp/131222827X",
    "https://www.uptodate.com/contents/etiology-of-hearing-loss-in-adults#!",
    "https://www.managedcaremag.com/archives/2016/2/economic-evaluation-linaclotide-treatment-adult-patients-chronic-idiopathic",
    "https://www.merriam-webster.com/dictionary/metaethics"

}
prefer_clean_domains = {x.split("/")[2] for x in prefer_clean}
prefer_plain_domains = {x.split("/")[2] for x in prefer_plain}

In [258]:
def clean_messed_up_utf_text(text):
    text = text.replace("\m", "m").replace("\l", "l").replace("\,", ",").replace("\c", "c").replace("\e", "e").replace("\i", "i").replace("\d", "d").replace("\M", "M").replace("\s", "s")
    return codecs.escape_decode(text)[0].decode("utf-8")


merged_texts = {}
different = set()
for idx, u in tqdm(enumerate(all_doc_ids), total=len(all_doc_ids)):
    if u in both and texts_in_plain[u].strip() != texts_in_clean[u].strip():
        domain = u.split("/")[2]
        if u in prefer_clean or domain in prefer_clean_domains:
            merged_texts[u] = clean_messed_up_utf_text(texts_in_clean[u])
            continue
        elif u in prefer_plain or domain in prefer_plain_domains:
            merged_texts[u] = clean_messed_up_utf_text(texts_in_plain[u])
            continue
        print(u)
        print("clean version:")
        print(clean_messed_up_utf_text(texts_in_clean[u]))
        print()
        print()
        print("plain version:")
        print(clean_messed_up_utf_text(texts_in_plain[u]))
        print("*********************************************************")
        break
    elif u in texts_in_clean:
        merged_texts[u] = clean_messed_up_utf_text(texts_in_clean[u])
    elif u in texts_in_plain:
        merged_texts[u] = clean_messed_up_utf_text(texts_in_plain[u])

  0%|          | 0/1074 [00:00<?, ?it/s]

/tmp/ipykernel_4062838/2431515496.py:3: DeprecationWarning: invalid escape sequence '\Ë'
  return codecs.escape_decode(text)[0].decode("utf-8")
/tmp/ipykernel_4062838/2431515496.py:3: DeprecationWarning: invalid escape sequence '\p'
  return codecs.escape_decode(text)[0].decode("utf-8")
/tmp/ipykernel_4062838/2431515496.py:3: DeprecationWarning: invalid escape sequence '\ '
  return codecs.escape_decode(text)[0].decode("utf-8")


In [262]:
with open("../data/merged_texts.tsv", 'w') as outf:
    for u in merged_texts:
        final_text = merged_texts[u].replace("\n", " ").strip()
        if "\n" in final_text:
            print("oops")
        outf.write(f"{u}\t{final_text}\n")
assert rawcount("../data/merged_texts.tsv") == len(metg